# Classification
Nous allons utiliser Un BBC news sport Dataset l'objectif est de créer un model qui detecte les news parlant du foot

# Notre flux de  travail
1- Lire Les documents


2- Préparez-les à la classification



3- Former un clas ificateur  (notre modèle)


4- Tester le modèle

## Création de la session spark

In [ ]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession
  .builder()
  .appName("Spark SQL basic example")
  .config("spark.some.config.option", "some-value")
  .getOrCreate()

// For implicit conversions like converting RDDs to DataFrames
import spark.implicits._

# Lire les  documents


Décompréssez le doc bbcsport.zip sur votre machine et utilisez le bon path pour le dossier

In [ ]:
val newsRDD = sc.wholeTextFiles("/path/bbcsport/football/*.txt")
val newsRDD1 = sc.wholeTextFiles("/path/bbcsport-fulltext/bbcsport/tennis/*.txt")

Création du type news

In [ ]:
case class news( path : String , text : String, sport : String , label : Double)

Création d un dataframe englobant les données

In [ ]:
val newdf1 = newsRDD.map(x => news(x._1 , x._2 , "foot", 1.0)).toDF()
newdf1.show()
val newdf2 = newsRDD1.map(x => news(x._1 , x._2 , "tennis", 0.0)).toDF()
newdf2.show()
import org.apache.spark.sql.functions.rand
val newdf3 = newdf1.union(newdf2).orderBy(rand())
newdf3.show()

Transformation du Fataframe en Dataset

In [ ]:
val newds= newdf3.as[news]
newds.show()

import des classe mllib et décomposition du dataset en données d'entrainement et données de test

In [ ]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row
val Array(training, test) = newds.randomSplit(Array[Double](0.7, 0.3), 12345)


In [ ]:
test.show()
test.count()
training.show()
training.count()

## Notre  pipeline
<img src="aa.png" ></img>

# transformateurs
<pre>

Tockenizer
Le processus de prise de texte (généralement sous forme de documents ou de  phrases) et de rupture de texte en termes (généralement des  mots)

HashingTF est un transformateur qui prend des ensembles de termes et convertit ces ensembles en vecteurs de longueur fixe.

</pre>

In [ ]:
val tokenizer = new Tokenizer()
  .setInputCol("text")
  .setOutputCol("words")

In [ ]:
val hashingTF = new HashingTF()
  .setNumFeatures(1000)
  .setInputCol(tokenizer.getOutputCol)
  .setOutputCol("features")

#  Estimateur que nous sommes en train d'utiliser
Régression logistique
<pre>
La Régression logistique est une méthode populaire pour prédire une réponse catégorique. elle est un cas particulier des modèles linéaires généralisés qui prédit la probabilité des résultats. Dans spark.ml, la régression logistique peut être utilisée pour prédire un  un résultat multiclasse en utilisant la régression logistique binomiale, ou elle peut être utilisé pour prédire un résultat multiclasse en utilisant la régression logistique  multinomiale. 
</pre>


In [ ]:
val lr = new LogisticRegression()
  .setMaxIter(10)
  .setRegParam(0.001)
val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, lr))

## Créer et ajuster  Pipeline


In [ ]:
// Fit the pipeline to training documents.
val model = pipeline.fit(training)

In [ ]:
// Now we can optionally save the fitted pipeline to disk
model.write.overwrite().save("/tmp/spark-logistic-regression-model")


In [ ]:
// We can also save this unfit pipeline to disk
pipeline.write.overwrite().save("/tmp/unfit-lr-model")

// And load it back in during production
val sameModel = PipelineModel.load("/tmp/spark-logistic-regression-model")


## Appliquer le model sur la data de test

In [ ]:
val predections = model.transform(test)
predections.select("path", "text", "sport","probability", "prediction")
  .collect()
  .foreach { case Row(path: String, text: String,sport:String, prob: Vector, prediction: Double) =>
    println(s"($sport) -->  prediction=$prediction")
  }

## Evaluer le model

In [ ]:
import  org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
val eval = new BinaryClassificationEvaluator() 
eval.evaluate(predections)